In [1]:
import pandas as pd

from sklearn.dummy import DummyRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("Dataset-V3_2.csv")

# Duplicating

In [3]:
data_dup = data[['Team2', 'Team1', 'Score2', 'Score1', 'Date/Time', 'nb_matches2', 'nb_matches1', 'goals_past2', 'goals_past1',
                 'ratio_goals_past2', 'ratio_goals_past1', 'nb_wins2', 'nb_wins1', 'ratio_wins2', 'ratio_wins1', 'nb_losses2',
                 'nb_losses1', 'ratio_losses2', 'ratio_losses1', 'nb_draws2', 'nb_draws1', 'ratio_draws2', 'ratio_draws1',
                 'nb_wins_opp2', 'nb_wins_opp1', 'ratio_wins_opp2', 'ratio_wins_opp1', 'nb_losses_opp2', 'nb_losses_opp1',
                 'ratio_losses_opp2', 'ratio_losses_opp1', 'nb_draws_opp2', 'nb_draws_opp1', 'ratio_draws_opp2',
                 'ratio_draws_opp1']]
data_dup.columns = ['Team1', 'Team2', 'Score1', 'Score2', 'Date/Time', 'nb_matches1', 'nb_matches2', 'goals_past1', 
                    'goals_past2', 'ratio_goals_past1', 'ratio_goals_past2', 'nb_wins1', 'nb_wins2', 'ratio_wins1',
                    'ratio_wins2', 'nb_losses1', 'nb_losses2', 'ratio_losses1', 'ratio_losses2', 'nb_draws1', 'nb_draws2', 
                    'ratio_draws1', 'ratio_draws2', 'nb_wins_opp1', 'nb_wins_opp2', 'ratio_wins_opp1',
                    'ratio_wins_opp2', 'nb_losses_opp1', 'nb_losses_opp2', 'ratio_losses_opp1', 'ratio_losses_opp2', 
                    'nb_draws_opp1', 'nb_draws_opp2', 'ratio_draws_opp1', 'ratio_draws_opp2']
data = data.append(data_dup, ignore_index=True)

C:\Users\ASUS\Anaconda3\envs\python\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


# Split into Train and Test

In [4]:
df_test = data.loc[data["Date/Time"] >= '2018']
df_train = data.loc[data["Date/Time"] < '2018']

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Get Target Vector

In [5]:
y_train = []
for i in range(len(df_train)):
    if df_train['Score1'][i]+df_train['Score2'][i] == 0:
        y_train.append(0)
    else:
        y_train.append(df_train['Score1'][i]/(df_train['Score1'][i]+df_train['Score2'][i]))
        
y_test = []
for i in range(len(df_test)):
    if df_test['Score1'][i] > df_test['Score2'][i]:
        y_test.append(1)
    elif df_test['Score1'][i] < df_test['Score2'][i]:
        y_test.append(2)
    elif df_test['Score1'][i] == df_test['Score2'][i]:
        y_test.append(0)

# Deleting Useless Columns: Getting X

In [6]:
X_train = df_train.drop(['Team1','Team2','Score1','Score2','Date/Time','date_minus_1year'],axis=1)
X_test = df_test.drop(['Team1','Team2','Score1','Score2','Date/Time','date_minus_1year'],axis=1)

In [7]:
cols = X_train.columns

# Scaling

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\ASUS\Anaconda3\envs\python\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ASUS\Anaconda3\envs\python\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\ASUS\Anaconda3\envs\python\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


# Trying out Regression Algorithms

In [9]:
def reg_to_class (y):
    y_class = []
    
    for i in range(len(y)//2):
        if abs(y[i] - y[i+len(y)//2]) < 0.05:
            y_class.append(0)
        elif y[i] > y[i+len(y)//2]:
            y_class.append(1)
        elif y[i] < y[i+len(y)//2]:
            y_class.append(2)
            
    y_class2 = []
    for i in y_class:
        if i == 0:
            y_class2.append(0)
        elif i == 1:
            y_class2.append(2)
        elif i == 2:
            y_class2.append(1)
        
    return(y_class+y_class2)

In [10]:
model = DummyRegressor()
model.fit(X_train, y_train)
accuracy_score(reg_to_class(model.predict(X_test)),y_test)

0.27450980392156865

In [11]:
print(classification_report(y_test, reg_to_class(model.predict(X_test))))

              precision    recall  f1-score   support

           0       0.27      1.00      0.43        84
           1       0.00      0.00      0.00       111
           2       0.00      0.00      0.00       111

   micro avg       0.27      0.27      0.27       306
   macro avg       0.09      0.33      0.14       306
weighted avg       0.08      0.27      0.12       306



C:\Users\ASUS\Anaconda3\envs\python\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
model = MLPRegressor(random_state=0)
model.fit(X_train, y_train)
accuracy_score(reg_to_class(model.predict(X_test)),y_test)

0.4444444444444444

In [13]:
model = GradientBoostingRegressor(random_state=0)
model.fit(X_train, y_train)
accuracy_score(reg_to_class(model.predict(X_test)),y_test)

0.43137254901960786

In [14]:
for i in range(len(cols)):
    print(cols[i], ' --> ',model.feature_importances_[i])

goals_past1  -->  0.10378995549193121
goals_past2  -->  0.019783605709638982
nb_draws1  -->  0.0022573639301378345
nb_draws2  -->  0.010155602575434958
nb_draws_opp1  -->  8.00101005443101e-06
nb_draws_opp2  -->  0.0007859827426925562
nb_losses1  -->  0.018132213121641746
nb_losses2  -->  0.031630092775952516
nb_losses_opp1  -->  0.0
nb_losses_opp2  -->  0.00039781456038505577
nb_matches1  -->  0.012095320750479128
nb_matches2  -->  0.024753101121018334
nb_wins1  -->  0.007787286739480891
nb_wins2  -->  0.03497671374870895
nb_wins_opp1  -->  8.647401760895753e-05
nb_wins_opp2  -->  5.326161354584175e-06
ratio_draws1  -->  0.005261521090580607
ratio_draws2  -->  0.007338137504706184
ratio_draws_opp1  -->  0.00032188668166780235
ratio_draws_opp2  -->  0.0002560106976991366
ratio_goals_past1  -->  0.29044960710833306
ratio_goals_past2  -->  0.3378333688584189
ratio_losses1  -->  0.03930466875174667
ratio_losses2  -->  0.04340549259696847
ratio_losses_opp1  -->  0.0005481986570341013
ratio

In [15]:
model = RandomForestRegressor(random_state=0)
model.fit(X_train, y_train)
accuracy_score(reg_to_class(model.predict(X_test)),y_test)

C:\Users\ASUS\Anaconda3\envs\python\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.3464052287581699

In [16]:
for i in range(len(cols)):
    print(cols[i], ' --> ',model.feature_importances_[i])

goals_past1  -->  0.07550455611713043
goals_past2  -->  0.07435339955474304
nb_draws1  -->  0.02169202216078449
nb_draws2  -->  0.020815376546162605
nb_draws_opp1  -->  0.004088587306886436
nb_draws_opp2  -->  0.004102451680082489
nb_losses1  -->  0.014939745491084655
nb_losses2  -->  0.015024868779249283
nb_losses_opp1  -->  0.0028826597926090913
nb_losses_opp2  -->  0.0031778500874876923
nb_matches1  -->  0.04877232620192847
nb_matches2  -->  0.04883611653421059
nb_wins1  -->  0.015279184264821185
nb_wins2  -->  0.0143564717822852
nb_wins_opp1  -->  0.003265264007770014
nb_wins_opp2  -->  0.002996595557452301
ratio_draws1  -->  0.051982908801523564
ratio_draws2  -->  0.04861535359457022
ratio_draws_opp1  -->  0.004907559217871648
ratio_draws_opp2  -->  0.004828086392187717
ratio_goals_past1  -->  0.1398602351244012
ratio_goals_past2  -->  0.14031902716017836
ratio_losses1  -->  0.058489055254796286
ratio_losses2  -->  0.05859471077975913
ratio_losses_opp1  -->  0.004602623444818458
r

In [17]:
model = AdaBoostRegressor(random_state=0)
model.fit(X_train, y_train)
accuracy_score(reg_to_class(model.predict(X_test)),y_test)

0.39869281045751637

In [18]:
for i in range(len(cols)):
    print(cols[i], ' --> ',model.feature_importances_[i])

goals_past1  -->  0.025814417721173266
goals_past2  -->  0.007700289716484332
nb_draws1  -->  0.0
nb_draws2  -->  0.0
nb_draws_opp1  -->  0.0
nb_draws_opp2  -->  0.0
nb_losses1  -->  0.0
nb_losses2  -->  0.0
nb_losses_opp1  -->  0.0
nb_losses_opp2  -->  0.0
nb_matches1  -->  0.0
nb_matches2  -->  0.0
nb_wins1  -->  0.0
nb_wins2  -->  0.0
nb_wins_opp1  -->  0.0
nb_wins_opp2  -->  0.0
ratio_draws1  -->  0.0
ratio_draws2  -->  0.0
ratio_draws_opp1  -->  0.0
ratio_draws_opp2  -->  0.0
ratio_goals_past1  -->  0.4467497493466612
ratio_goals_past2  -->  0.5008912947695241
ratio_losses1  -->  0.0
ratio_losses2  -->  0.018844248446156984
ratio_losses_opp1  -->  0.0
ratio_losses_opp2  -->  0.0
ratio_wins1  -->  0.0
ratio_wins2  -->  0.0
ratio_wins_opp1  -->  0.0
ratio_wins_opp2  -->  0.0


In [19]:
model = BaggingRegressor(random_state=0)
model.fit(X_train, y_train)
accuracy_score(reg_to_class(model.predict(X_test)),y_test)

0.37254901960784315

In [20]:
model = TransformedTargetRegressor()
model.fit(X_train, y_train)
accuracy_score(reg_to_class(model.predict(X_test)),y_test)

0.41830065359477125